In [1]:
import os
import tensorflow as tf
import wget
from zipfile import PyZipFile
import shutil
import pandas as pd
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array
import cv2
import h5py
import glob
import keras


In [2]:
#Directories for Data
temp_directory = 'data/temp'
orig_dir = 'data/images'
train_directory = 'data/train'
val_dir = 'data/val'
aug_images_directory = train_directory + '/augumented_images'


In [3]:
drivFace_images = "C:/Users/selloh/Desktop/Datasets/DrivFace/DrivImages"
# drivFace_images = './datasets/Drivface/DrivImages'
# drivFace_annotations = './datasets/DrivFace/drivPoints.txt'
drivFace_annotations = 'C:/Users/selloh/Desktop/Datasets/DrivFace/drivPoints.txt'


In [4]:

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00378/DrivFace.zip'
# File name of the output of pre processing
hdf5_path = 'DrivFace.h5'
# Shape of input data
HEIGHT = 224
WIDTH = 224
CHANNELS = 3
SHAPE = (HEIGHT, WIDTH, CHANNELS)


In [5]:
hdf5_path = 'DrivFace.h5'


In [6]:
# face_detector = dlib.get_frontal_face_detector()
# landmark_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
def load_dataset():
  if not os.path.exists(train_directory):
    os.makedirs(temp_directory)
    os.makedirs(train_directory)
    print("Downloading dataset to "+ temp_directory)
    file = wget.download(url, out=temp_directory)
    print("\nUnzipping the files..")
    pzf = PyZipFile(file)
    pzf.extractall(temp_directory)
    pzf = PyZipFile(temp_directory+'/DrivFace/DrivImages.zip')
    pzf.extractall(temp_directory)
    print("Moving files to "+train_directory)
    for file in os.listdir(temp_directory+'/DrivImages'):
      shutil.move(temp_directory+'/DrivImages/'+file,train_directory+'/'+file)
    shutil.move(temp_directory+'/DrivFace/drivPoints.txt',train_directory+'/drivPoints.txt')
    print("Deleting temporary directory "+ temp_directory)
    shutil.rmtree(temp_directory)


In [7]:
def load_annotations(image_path):
    base_name = os.path.basename(image_path).split('.')[0]
    annotations = pd.read_csv(drivFace_annotations)
    annotation = annotations[annotations['fileName'].str.contains(base_name)].iloc[0]

    landmarks = {
        "xF": annotation['xF'], "yF": annotation['yF'], "wF": annotation['wF'], "hF": annotation['hF'],
        "xRE": annotation['xRE'], "yRE": annotation['yRE'],
        "xLE": annotation['xLE'], "yLE": annotation['yLE'],
        "xN": annotation['xN'], "yN": annotation['yN'],
        "xRM": annotation['xRM'], "yRM": annotation['yRM'],
        "xLM": annotation['xLM'], "yLM": annotation['yLM']
    }

    pose = annotation['label']

    return landmarks, pose



In [8]:
def create_augumented_images(directory):
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        fill_mode='nearest')
    if not os.path.exists(directory):
        os.makedirs(directory)
    else:
        return
    print("Generating augumented images in the directory " + directory)
    for index, row in df_label.iterrows():
            file = row['fileName'] + ".jpg"
            image = cv2.imread(train_directory + '/' + file)
            y = int(row['yF'])
            x = int(row['xF'])
            w = int(row['wF'])
            h = int(row['hF'])
            #image = image[y:y+h, x:x+w]
            image = cv2.resize(image, (HEIGHT, WIDTH), interpolation = cv2.INTER_AREA)
            X = img_to_array(image)
            X = X.reshape((1,) + X.shape)
            i = 0
            for batch in datagen.flow(X, batch_size=1,
                              save_to_dir=directory, save_prefix=file, save_format='jpg'):
                i += 1
                if i > 20:
                    break  # otherwise the generator would loop indefinitely



In [9]:
def proc_images():
    num_images = 0
    dataset_X = []
    dataset_y = []
    if os.path.isfile(hdf5_path):
        return
    else:
        print("writing images to " + hdf5_path)
        with h5py.File(hdf5_path, mode='w') as hf:
            for index, row in df_label.iterrows():
                file = row['fileName'] + ".jpg"
                image = cv2.imread(train_directory + '/' + file)
                y = int(row['yF'])
                x = int(row['xF'])
                w = int(row['wF'])
                h = int(row['hF'])
                #image = image[y:y+h, x:x+w]
                image = cv2.resize(image, (HEIGHT, WIDTH), interpolation = cv2.INTER_AREA)
                dataset_X.append(image)
                dataset_y.append(row['label'])
                num_images = num_images + 1
                for augumented_image in glob.glob(train_directory + '/augumented_images/' + file + "*"):
                    image = cv2.imread(augumented_image)
                    y = int(row['yF'])
                    x = int(row['xF'])
                    w = int(row['wF'])
                    h = int(row['hF'])
                    #image = image[y:y+h, x:x+w]
                    image = cv2.resize(image, (HEIGHT, WIDTH), interpolation = cv2.INTER_AREA)
                    dataset_X.append(image)
                    dataset_y.append(row['label'])
                    num_images = num_images + 1
            dataset_X = hf.create_dataset(
                    name='dataset_X',
                    data=dataset_X,
                    shape=(len(dataset_X), HEIGHT, WIDTH, CHANNELS),
                    maxshape=(len(dataset_X), HEIGHT, WIDTH, CHANNELS),
                    compression="gzip",
                    compression_opts=9)
            dataset_y = hf.create_dataset(
                    name='dataset_y',
                    data=dataset_y,
                    shape=(len(dataset_y), 1,),
                    maxshape=(len(dataset_y), None,),
                    compression="gzip",
                    compression_opts=9)
            number_images = next(os.walk(train_directory))[2]
            number_images_aug = next(os.walk(aug_images_directory))[2]
            print("Number of images written to "+ hdf5_path + " is: " + str(len(number_images)+len(number_images_aug)))

In [10]:
load_dataset()

In [11]:
df_label = pd.read_csv(train_directory+'/drivPoints.txt')
df_label


,fileName,subject,imgNum,label,ang,xF,yF,wF,hF,xRE,yRE,xLE,yLE,xN,yN,xRM,yRM,xLM,yLM
0,20130529_01_Driv_001_f,1,1,2,0,292,209,100,112,323,232,367,231,353,254,332,278,361,278
1,20130529_01_Driv_002_f,1,2,2,0,286,200,109,128,324,235,366,235,353,258,333,281,361,281
2,20130529_01_Driv_003_f,1,3,2,0,290,204,105,121,325,240,367,239,351,260,334,282,362,282
3,20130529_01_Driv_004_f,1,4,2,0,287,202,112,118,325,230,369,230,353,253,335,274,362,275
4,20130529_01_Driv_005_f,1,5,2,0,290,193,104,119,325,224,366,225,353,244,333,268,363,268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,20130530_04_Driv_086_f,4,86,2,-15,278,183,128,141,307,218,354,210,330,247,324,273,356,266
602,20130530_04_Driv_087_lr,4,87,1,-30,268,186,128,134,296,222,344,212,319,247,316,274,347,269
603,20130530_04_Driv_088_lr,4,88,1,-30,264,187,127,131,287,220,334,211,304,247,305,272,337,270
604,20130530_04_Driv_089_f,4,89,2,-15,264,175,143,136,295,207,345,200,320,234,314,261,351,251


In [12]:
df_label = df_label.dropna()

In [13]:
create_augumented_images(aug_images_directory)

In [14]:
proc_images()